In [1]:
import numpy as np
import pandas as pd
from pandas import DataFrame as df
from scipy.stats import multivariate_normal
from scipy import integrate
from scipy.optimize import minimize
import time

# Q1. Sample Construction

In [2]:
# Draw X 
x = np.random.uniform(0,1,100)

In [3]:
# Draw Epsilon
cov = np.array([[1, 0.3],
                [0.3, 1]])
A = np.linalg.cholesky(cov)

epsilon_1 = []
epsilon_2 = []
temp_1 = np.random.standard_normal(100)
temp_2 = np.random.standard_normal(100)

for i in range(100):
    temp = np.array([[temp_1[i]],[temp_2[i]]])
    result = A@temp
    epsilon_1.append(result[0][0])
    epsilon_2.append(result[1][0])

In [4]:
# Generate sample data
x_1 = -2*x 
x_2 = -1*x

u_1 = 1 + x_1 + np.array(epsilon_1)
u_2 = -0.5 + x_2 + np.array(epsilon_2)
u_3 = np.array([0 for i in range(100)])

d_1 = []
d_2 = []
d_3 = []
for i in range(100):
    highest = np.array([u_1[i], u_2[i], u_3[i]]).max()
    if highest == u_1[i]:
        d_1.append(1)
        d_2.append(0)
        d_3.append(0)
    elif highest == u_2[i]:
        d_1.append(0)
        d_2.append(1)
        d_3.append(0)
    else:
        d_1.append(0)
        d_2.append(0)
        d_3.append(1)

In [5]:
index = [i for i in range(100)]
data = df({'i': index,
           'u_1': u_1,
           'u_2': u_2,
           'u_3': u_3,
           'd_1': d_1,
           'd_2': d_2,
           'd_3': d_3,
           'x_1': x_1,
           'x_2': x_2})
data.head(5)

,i,u_1,u_2,u_3,d_1,d_2,d_3,x_1,x_2
0,0,0.740906,-1.062944,0,1,0,0,-1.100575,-0.550287
1,1,-0.568280,-1.743027,0,0,0,1,-1.306819,-0.653409
2,2,-0.271927,-2.100951,0,0,0,1,-0.973817,-0.486909
3,3,0.852650,-0.047383,0,1,0,0,-0.148482,-0.074241
4,4,-2.690505,-2.182586,0,0,0,1,-1.754038,-0.877019


In [6]:
print("proportion of individuals choosing 1: ", np.array(d_1).sum()/100)
print("proportion of individuals choosing 2: ", np.array(d_2).sum()/100)
print("proportion of individuals choosing 3: ", np.array(d_3).sum()/100)

proportion of individuals choosing 1:  0.47
proportion of individuals choosing 2:  0.08
proportion of individuals choosing 3:  0.45


# Q2. MLE

In [7]:
def log_likelihood(theta, data):
    '''
    input: data frame
           theta <-- order: alpha_1, alpha_2, beta, gamma_1, gamma_2, gamma_3
    output: log likelihood 
    '''
    gamma = np.array([[theta[3], 0],
                      [theta[5], theta[4]]])
    cov_matrix = gamma@gamma.transpose()
    bivar_normal = multivariate_normal(mean = [0, 0], cov = cov_matrix)
    
    P_1 = []
    P_2 = []
    alpha_1 = theta[0] 
    alpha_2 = theta[1]
    beta = theta[2]
    x_1 = data['x_1']
    x_2 = data['x_2']

    for i in range(100):
        integrand_1 = lambda epsilon_1, epsilon_2: int(alpha_1 + beta*x_1[i] + epsilon_1 >= \
                                                       alpha_2 + beta*x_2[i] + epsilon_2)*\
                                                   int(alpha_1 + beta*x_1[i] + epsilon_1 >= 0)*\
                                                   bivar_normal.pdf([epsilon_1, epsilon_2])
        integrand_2 = lambda epsilon_1, epsilon_2: int(alpha_1 + beta*x_1[i] + epsilon_1 <= \
                                                       alpha_2 + beta*x_2[i] + epsilon_2)*\
                                                   int(alpha_2 + beta*x_2[i] + epsilon_2 >= 0)*\
                                                   bivar_normal.pdf([epsilon_1, epsilon_2])
        result_1 = integrate.dblquad(integrand_1, -np.inf, np.inf, -np.inf, np.inf, epsabs=1.0e-2)
        result_2 = integrate.dblquad(integrand_2, -np.inf, np.inf, -np.inf, np.inf, epsabs=1.0e-2)
        P_1.append(result_1[0])
        P_2.append(result_2[0])
    
    P_1 = np.array(P_1)
    P_2 = np.array(P_2)
    P_3 = 1 - P_1 - P_2
        
    log_likelihood = (np.array(data['d_1'])*P_1 + np.array(data['d_2'])*P_2 + np.array(data['d_3'])*P_3).sum()
    return -log_likelihood

In [8]:
start_time = time.time()
MLE_result = minimize(log_likelihood, [1.01, -0.48, 0.99, 1, 0.9539392, 0.3], args = (data), method='Nelder-Mead', options={'maxiter':5})
print("--- %s seconds for MLE---" % (time.time() - start_time))

--- 2488.315544605255 seconds for MLE---


# Q3. SMM

In [9]:
def smm(theta, data):
    alpha_1 = theta[0] 
    alpha_2 = theta[1]
    beta = theta[2]
    x_1 = data['x_1']
    x_2 = data['x_2']

    #calculate probability
    gamma = np.array([[theta[3], 0],
                  [theta[5], theta[4]]])
    P_1 = np.array([0 for i in range(100)])
    P_2 = np.array([0 for i in range(100)])
    for s in range(10):
        eta_1 = np.random.standard_normal(100)
        eta_2 = np.random.standard_normal(100)
        
        epsilon_1 = []
        epsilon_2 = []
        for i in range(100):
            eta = np.array([[eta_1[i]], [eta_2[i]]])
            result = gamma@eta
            epsilon_1.append(result[0][0])
            epsilon_2.append(result[1][0])
        
        temp1 = []
        temp2 = []
        for i in range(100):
            indicator_1 = int(alpha_1 + beta*x_1[i] + epsilon_1[i] >= alpha_2 + beta*x_2[i] + epsilon_2[i])*\
                          int(alpha_1 + beta*x_1[i] + epsilon_1[i] >= 0)
            indicator_2 = int(alpha_2 + beta*x_2[i] + epsilon_2[i] >= alpha_1 + beta*x_1[i] + epsilon_1[i])*\
                          int(alpha_2 + beta*x_2[i] + epsilon_2[i] >= 0)
            temp1.append(indicator_1)
            temp2.append(indicator_2)
        
        temp1 = np.array(temp1)
        temp2 = np.array(temp2)
        
        P_1 = P_1 + temp1
        P_2 = P_2 + temp2
    
    P_1 = P_1/10
    P_2 = P_2/10
    P_3 = 1 - P_1 - P_2

    #calculate objective value
    P = [P_1, P_2, P_3]
    d_1 = list(data['d_1'])
    d_2 = list(data['d_2'])
    d_3 = list(data['d_3'])
    d = [d_1, d_2, d_3]
    
    smm_obj = 0
    for j in range(2):
        left = np.array([[0,0]])
        right = left.transpose()

        for i in range(100):
            scalar = d[j][i] - P[j][i]
            left_vector = np.array([[1,x[i]]])
            left = left + scalar*left_vector
            right_vector = left_vector.transpose()
            right = right + scalar*right_vector

    result = left@right
    smm_obj = smm_obj + result[0][0]
            
    return smm_obj

In [10]:
start_time = time.time()
SMM_result = minimize(smm, [1.01, -0.48, 0.99, 1, 0.9539392, 0.3], args = (data), method='Nelder-Mead', options={'maxiter':300})
print("--- %s seconds for SMM---" % (time.time() - start_time))

--- 12.799067974090576 seconds for SMM---


# Q4. Smooth SMM

In [11]:
def ssmm(theta, data):
    alpha_1 = theta[0] 
    alpha_2 = theta[1]
    beta = theta[2]
    gamma_1 = theta[3]
    gamma_2 = theta[4]
    gamma_3 = theta[5]
    x_1 = data['x_1']
    x_2 = data['x_2']
    
    sigma_1 = np.array([[(gamma_1 - gamma_3)**2 + gamma_2**2, gamma_1*(gamma_1-gamma_3)],
                        [gamma_1*(gamma_1-gamma_3), gamma_1**2]])
    sigma_2 = np.array([[(gamma_1 - gamma_3)**2 + gamma_2**2, gamma_2**2 - gamma_3*(gamma_1 - gamma_3)],
                        [gamma_2**2 - gamma_3*(gamma_1 - gamma_3), gamma_1**2]])
    
    P_1 = np.array([0 for i in range(100)])
    P_2 = np.array([0 for i in range(100)])
    for s in range(10):
        u_12 = np.random.exponential(scale = 1, size = 100)
        u_10 = np.random.exponential(scale = 1, size = 100)
        u_21 = np.random.exponential(scale = 1, size = 100)
        u_20 = np.random.exponential(scale = 1, size = 100)
        
        temp1 = []
        temp2 = []
        for i in range(100):
            mu_1 = [alpha_1 - alpha_2 + beta*(x_1[i]-x_2[i]), alpha_1 + beta*x_1[i]]
            mu_2 = [alpha_2 - alpha_1 + beta*(x_2[i]-x_1[i]), alpha_2 + beta*x_2[i]]
            bivar_normal_1 = multivariate_normal(mean = mu_1, cov = sigma_1)
            bivar_normal_2 = multivariate_normal(mean = mu_2, cov = sigma_2)
            
            numerator_1 = bivar_normal_1.pdf([u_12[i], u_10[i]])
            numerator_2 = bivar_normal_2.pdf([u_21[i], u_20[i]])
            
            denominator_1 = np.exp(-u_12[i])*np.exp(-u_10[i])
            denominator_2 = np.exp(-u_21[i])*np.exp(-u_20[i])
            
            temp1.append(numerator_1/denominator_1)
            temp2.append(numerator_2/denominator_2)
        
        temp1 = np.array(temp1)
        temp2 = np.array(temp2)
        
        P_1 = P_1 + temp1
        P_2 = P_2 + temp2

    P_1 = P_1/10
    P_2 = P_2/10
    P_3 = 1 - P_1 - P_2

    #calculate objective value
    P = [P_1, P_2, P_3]
    d_1 = list(data['d_1'])
    d_2 = list(data['d_2'])
    d_3 = list(data['d_3'])
    d = [d_1, d_2, d_3]
    
    ssmm_obj = 0
    for j in range(2):
        left = np.array([[0,0]])
        right = left.transpose()

        for i in range(100):
            scalar = d[j][i] - P[j][i]
            left_vector = np.array([[1,x[i]]])
            left = left + scalar*left_vector
            right_vector = left_vector.transpose()
            right = right + scalar*right_vector

    result = left@right
    ssmm_obj = ssmm_obj + result[0][0]
            
    return ssmm_obj

In [12]:
start_time = time.time()
SSMM_result = minimize(ssmm, [1.01, -0.48, 0.99, 1, 0.9539392, 0.3], args = (data), method='Nelder-Mead', options={'maxiter':300})
print("--- %s seconds for smoothSMM---" % (time.time() - start_time))

--- 171.50476717948914 seconds for smoothSMM---


In [15]:
method = ['MLE', 'SMM', 'sSMM']
alpha_1 = [MLE_result.x[0], SMM_result.x[0], SSMM_result.x[0]]
alpha_2 = [MLE_result.x[1], SMM_result.x[1], SSMM_result.x[1]]
beta = [MLE_result.x[2], SMM_result.x[2], SSMM_result.x[2]]
gamma_1 = [MLE_result.x[3], SMM_result.x[3], SSMM_result.x[3]]
gamma_2 = [MLE_result.x[4], SMM_result.x[4], SSMM_result.x[4]]
gamma_3 = [MLE_result.x[5], SMM_result.x[5], SSMM_result.x[5]]

result_table = df({'method': method,
                   'alpha_1': alpha_1,
                   'alpha_2': alpha_2,
                   'beta': beta,
                   'gamma_1': gamma_1,
                   'gamma_2': gamma_2,
                   'gamma_3': gamma_3})

In [16]:
result_table

,method,alpha_1,alpha_2,beta,gamma_1,gamma_2,gamma_3
0,MLE,1.077333,-0.512000,1.056000,0.988889,0.869145,0.282500
1,SMM,1.009931,-0.494967,1.011073,1.011327,0.951173,0.307479
2,sSMM,1.030574,-0.463667,0.995729,1.020370,0.968734,0.306111
